In [2]:
#import necessary libraries
import tensorflow as tf
import tensorflow_datasets as tfds

2025-04-27 20:42:16.737302: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/maxwatzky/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
#load the mnist numbers dataset
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [4]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [5]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
#IMPORTANT - WE DESIGN THE NEURAL NETWORK WITH THE FOLLOWING STRUCTURE:
  #INPUT LAYER - 128 NEURONS
  #HIDDEN LAYER - 64 NEURONS
  #HIDDEN LAYER - 28 NEURONS 
  #OUTPUT LAYER - 10 NEURONS

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(28, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 4s 3ms/step - loss: 0.3383 - sparse_categorical_accuracy: 0.8996 - val_loss: 0.1451 - val_sparse_categorical_accuracy: 0.9552
Epoch 2/6
469/469 [==============================] - 1s 2ms/step - loss: 0.1226 - sparse_categorical_accuracy: 0.9633 - val_loss: 0.1286 - val_sparse_categorical_accuracy: 0.9588
Epoch 3/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0849 - sparse_categorical_accuracy: 0.9740 - val_loss: 0.0951 - val_sparse_categorical_accuracy: 0.9691
Epoch 4/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0659 - sparse_categorical_accuracy: 0.9798 - val_loss: 0.0884 - val_sparse_categorical_accuracy: 0.9748
Epoch 5/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0518 - sparse_categorical_accuracy: 0.9840 - val_loss: 0.0795 - val_sparse_categorical_accuracy: 0.9767
Epoch 6/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0413 - sparse_categorical_accu

In [ ]:
# params will look like:
# [ W_flatten→Dense1, b_flatten→Dense1,
#   W_Dense1→Dense2,   b_Dense1→Dense2,
#   W_Dense2→Dense3,   b_Dense2→Dense3,
#   W_Dense3→Dense4,   b_Dense3→Dense4,
#   W_Dense4→output,   b_Dense4→output ]


import numpy as np
params = model.get_weights()

In [58]:
#can't save directly, since each layer has a different size
#gotta get creative
for i in range(len(params)):
    type_string = 'weights' * (i%2 == 0) + 'biases' * (i%2 == 1)
    np.save("nn_" + type_string + "_layer_" + str(int(i/2)) + ".npy", params[i])